<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/KG_Enhanced_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain langchain-community faiss-cpu kuzu pyvis
%pip install -qU sentence-transformers torch plotly pandas scikit-learn networkx
%pip install -qU torch torchvision
%pip install -qU langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.

In [2]:
import kuzu
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import networkx as nx
import plotly.graph_objects as go
import plotly.express as px
from sklearn.manifold import TSNE
import numpy as np
import pandas as pd

In [3]:
from transformers import pipeline

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model="llama-3.1-70b-versatile",
    api_key="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"
)

In [6]:
system = "You are a helpful assistant."
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | llm

In [7]:
chain.invoke({"text": "Explain the importance of low latency for LLMs."})

AIMessage(content="Low latency is crucial for Large Language Models (LLMs) because it directly impacts the user experience and the overall performance of the model. Here are some reasons why low latency is important for LLMs:\n\n1. **Responsiveness**: LLMs are typically used in applications where users expect a rapid response, such as chatbots, virtual assistants, and language translation systems. Low latency ensures that the model responds quickly to user input, making the interaction feel more natural and engaging.\n2. **Conversational Flow**: In conversational AI applications, low latency is essential to maintain a smooth conversation flow. If the model takes too long to respond, the conversation can feel disjointed and unnatural, leading to a poor user experience.\n3. **Real-time Processing**: Many LLMs are used in real-time applications, such as language translation, sentiment analysis, and text summarization. Low latency ensures that the model can process and respond to user inpu

In [8]:
# Initialize Kuzu DB
db = kuzu.Database("my_knowledge_graph")
conn = kuzu.Connection(db)

In [9]:
# Create schema for the graph
conn.execute("CREATE NODE TABLE Entity (name STRING, PRIMARY KEY (name))")
conn.execute("CREATE REL TABLE Relation (FROM Entity TO Entity, predicate STRING)")

In [10]:
websites = [
    "https://neurons-lab.com/",
    "https://neurons-lab.com/about-us/",
    "https://www.crunchbase.com/organization/neurons-lab",
]

In [11]:
loader = WebBaseLoader(websites)
documents = loader.load()

In [12]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

In [13]:
vectorstore = FAISS.from_documents(texts, embeddings)

In [14]:
# Knowledge extraction and graph population (same as before)
kg_triple_extract_template = """
Extract up to 5 knowledge triplets from the text below in the form (subject, predicate, object).
Text: {text}
Triplets:
"""
kg_triple_extract_prompt = PromptTemplate(
    input_variables=["text"],
    template=kg_triple_extract_template,
)

kg_triple_extract_chain = LLMChain(llm=llm, prompt=kg_triple_extract_prompt)

for text in texts:
    triplets = kg_triple_extract_chain.run(text.page_content)
    for triplet in triplets.split('\n'):
        if triplet.strip():
            try:
                subject, predicate, obj = eval(triplet.strip())
                conn.execute("INSERT INTO Entity (name) VALUES ($1) ON CONFLICT DO NOTHING", [subject])
                conn.execute("INSERT INTO Entity (name) VALUES ($1) ON CONFLICT DO NOTHING", [obj])
                conn.execute("INSERT INTO Relation VALUES ($1, $2, $3)", [subject, obj, predicate])
            except:
                print(f"Failed to process triplet: {triplet}")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Failed to process triplet: Here are 5 knowledge triplets extracted from the text:
Failed to process triplet: 1. (Neurons Lab, is a, AI solution development expert)
Failed to process triplet: 2. (Neurons Lab, partners with, fast-growing companies)
Failed to process triplet: 3. (Neurons Lab, has expertise in, HealthTech, CleanTech, and RetailTech)
Failed to process triplet: 4. (Neurons Lab, is a, AWS Certified Advanced partner)
Failed to process triplet: 5. (Neurons Lab, offers, AI feasibility analysis, AI solution engineering, and AI operations management)
Failed to process triplet: Note: The triplets are in the form (subject, predicate, object), where the subject is the entity that the sentence is about, the predicate is the action or relation, and the object is the entity that the action or relation is applied to.
Failed to process triplet: Here are 5 knowledge triplets extracted from the text:
Failed to process triplet: 1. (Magellan X, is related to, Cleantech)
Failed to process trip

<string>:1: SyntaxWarning: 'float' object is not callable; perhaps you missed a comma?


Failed to process triplet: Here are 5 knowledge triplets extracted from the text:
Failed to process triplet: 1. (Our expertise, lies at the intersection of, AI, advanced science, and business)
Failed to process triplet:    *Note: This one is a bit tricky, as it's not a traditional subject-predicate-object structure. I'll try to break it down into a more traditional format:
Failed to process triplet:    (Our expertise, is related to, AI, advanced science, and business)
Failed to process triplet:    *However, I'll keep the original format for consistency.
Failed to process triplet: 2. (Talent Pool, has, over 500 highly skilled individuals)
Failed to process triplet: 3. (Our teams, consist of, PhD-level applied scientists, recognized DS/ML/AI Engineers, and MLOps)
Failed to process triplet: 4. (Our partnership, is, advanced-tier AWS partnership)
Failed to process triplet: 5. (Our solutions, include, NLP, computer vision, time series, and Recommender systems)
Failed to process triplet: Not

In [20]:
def get_graph_data():
    nodes_result = conn.execute("MATCH (e:Entity) RETURN e.name")
    edges_result = conn.execute("MATCH (e1:Entity)-[r:Relation]->(e2:Entity) RETURN e1.name, r.predicate, e2.name")

    nodes = [row[0] for row in nodes_result]
    edges = [(row[0], row[1], row[2]) for row in edges_result]

    return nodes, edges

In [21]:
# Enhanced graph visualization using Plotly
def visualize_graph_plotly():
    nodes, edges = get_graph_data()
    G = nx.Graph()

    for node in nodes:
        G.add_node(node[0])

    for edge in edges:
        G.add_edge(edge[0], edge[2], label=edge[1])

    pos = nx.spring_layout(G)

    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_x = [pos[node][0] for node in G.nodes()]
    node_y = [pos[node][1] for node in G.nodes()]

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2))

    node_adjacencies = []
    node_text = []
    for node, adjacencies in G.adjacency():
        node_adjacencies.append(len(adjacencies))
        node_text.append(f'{node}<br># of connections: {len(adjacencies)}')

    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='Knowledge Graph',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20,l=5,r=5,t=40),
                        annotations=[ dict(
                            text="",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002 ) ],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                    )

    fig.show()

In [22]:
# Embedding visualization
def visualize_embeddings():
    # Get embeddings
    doc_embeddings = [embeddings.embed_query(text.page_content) for text in texts]

    # Reduce dimensionality for visualization
    tsne = TSNE(n_components=3, random_state=42)
    vis_dims = tsne.fit_transform(doc_embeddings)

    # Create a DataFrame for Plotly
    df = pd.DataFrame(vis_dims, columns=['x', 'y', 'z'])
    df['text'] = [text.page_content[:100] + '...' for text in texts]  # Truncate text for readability

    # Create 3D scatter plot
    fig = px.scatter_3d(df, x='x', y='y', z='z', hover_data=['text'],
                        title='Document Embeddings Visualization')
    fig.show()

In [23]:
# Query function (same as before)
def query_graph(query):
    docs = vectorstore.similarity_search(query, k=2)
    context = "\n".join([doc.page_content for doc in docs])

    query_template = """
    Given the following context and question, provide a concise answer:
    Context: {context}
    Question: {question}
    Answer:
    """
    query_prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=query_template,
    )
    query_chain = LLMChain(llm=llm, prompt=query_prompt)

    return query_chain.run({"context": context, "question": query})

In [24]:
# Main execution
if __name__ == "__main__":
    visualize_graph_plotly()
    visualize_embeddings()

    # Example query
    question = "How can Neurons Lab help with a fintech use case to solve fraud?"
    answer = query_graph(question)
    print(f"Question: {question}\nAnswer: {answer}")

    # Close the database connection
    conn.close()

TypeError: 'QueryResult' object is not iterable